In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.applications import VGG16

In [2]:
(train_set, test_set), dataset_info = tfds.load('cats_vs_dogs', split=['train[:81%]', 'train[80%:]'], as_supervised=True, with_info=True)

In [3]:
def preprocess(img, label):
    img = tf.image.resize(img, (150, 150))
    img = img / 255.0
    return img, label

train_set = train_set.map(preprocess).batch(32).shuffle(buffer_size=1000).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
test_set = test_set.map(preprocess).batch(32).shuffle(buffer_size=1000).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [4]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))


In [5]:
# Freeze the layers

for layer in base_model.layers:
    layer.trainable = False

In [6]:
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout
from tensorflow.keras import Model

input_layer = Input(shape=(150, 150, 3))
x = base_model(input_layer, training=False)
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
output_layer = Dense(2, activation='softmax')(x)

In [7]:
model = Model(inputs=input_layer, outputs=output_layer)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 512)               4194816   
                                                                 
 dense_1 (Dense)             (None, 2)                 1026      
                                                                 
Total params: 18,910,530
Trainable params: 4,195,842
Non-trainable params: 14,714,688
_________________________________________________________________


In [8]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(train_set, epochs=5, verbose=1, validation_data=test_set)

Epoch 1/5


In [ ]:
test_loss, test_accuracy = model.evaluate(test_set)
model.save('cats_dogs_classifier.h5')

Classifer with VGG16 works perfectly fine

Lets try same architecture but without communication between two hemi

In [ ]:
input_layer = Input(shape=(150, 150, 3))
flatten_layer = Flatten()(input_layer)

# Left Buffer
buffer_input = Dense(512, activation='relu')(flatten_layer)
buffer_mid = Dense(1024, activation='relu')(buffer_input)
buffer_out = Dense(512, activation='relu')(buffer_mid)

hemi_mid = Dense(1024, activation="relu")(hemi_input)
hemi_mid = Dropout(0.3)(left_hemi_mid)
hemi_mid = Dense(512, activation="relu")(hemi_mid)
hemi_mid = Dropout(0.3)(left_hemi_mid)
hemi_mid = Dense(256, activation="relu")(hemi_mid)
hemi_mid = Dropout(0.3)(left_hemi_mid)
hemi_mid = Dense(64, activation="relu")(hemi_mid)
hemi_out = Dense(2, activation="sigmoid", name='left_out')(hemi_mid)